In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cross_validation, linear_model, preprocessing
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from __future__ import division
%matplotlib inline

ImportError: No module named sklearn

In [ ]:
raw_data = pd.read_csv('./learndata-latest.csv')
raw_data.head()

In [ ]:
crowd_labels = pd.read_csv('./crowdsource-20160208.csv')
crowd_labels.head()

In [ ]:
raw_data2 = pd.DataFrame()

True_disagree = 0. 
False_disagree = 0. 

NoConn = sum(raw_data['connected'])
NotConn = len(raw_data) - NoConn + 0. 

for i in range(0,len(crowd_labels)):
    temp = raw_data[raw_data['id_i']==crowd_labels['swi_id'][i]]
    temp2 = temp[temp['id_j']==crowd_labels['swj_id'][i]]
    raw_data2 = raw_data2.append(temp2,ignore_index=True)
    

    if (i%1000==0):
        print i
          

In [ ]:
raw_data2.head()
NoDis = sum(raw_data2['connected']!=crowd_labels['connected'])
print NoDis
raw_data2['connected']=crowd_labels['connected']

In [ ]:
# Remove features that we shouldn't learn (encoded geometries and sidewalk ID numbers)
near_line = raw_data2['near_line']  # Save for later
del raw_data2['near_line']
del raw_data2['id_i']
del raw_data2['id_j']

In [ ]:
# Binarizes categorical variables 
# (e.g. if 3 categories, makes 3 cols with 1s and 0s)
# X = pd.get_dummies(raw_data)  

# Turn categorical variables into integer labels
label_encoder = preprocessing.LabelEncoder()
raw_data2['curbtype_i'] = label_encoder.fit_transform(raw_data2['curbtype_i'])
raw_data2['curbtype_j'] = label_encoder.fit_transform(raw_data2['curbtype_j'])
raw_data2['side_i'] = label_encoder.fit_transform(raw_data2['side_i'])
raw_data2['side_j'] = label_encoder.fit_transform(raw_data2['side_j'])
raw_data2['surftype_i'] = label_encoder.fit_transform(raw_data2['surftype_i'])
raw_data2['surftype_j'] = label_encoder.fit_transform(raw_data2['surftype_j'])
raw_data2['intersects'] = label_encoder.fit_transform(raw_data2['intersects'])

X = raw_data2

X['same_curbtype'] = (X['curbtype_i'] == X['curbtype_j']).astype(int)
X['same_surftype'] = (X['surftype_i'] == X['surftype_j']).astype(int)
X['same_block'] = (X['bid_i'] == X['bid_j']).astype(int)
X['same_side'] = (X['side_i'] == X['side_j']).astype(int)

# TODO: There's probably more features we can generate with these two lengths
X['length_diff'] = abs(X['length_i'] - X['length_j'])
X['sw_width_diff'] = abs(X['sw_width_i'] - X['sw_width_j'])

y = X['connected']
X = X.drop('connected', 1)
X = X.drop('curbtype_i', 1)
X = X.drop('curbtype_j', 1)
X = X.drop('surftype_i', 1)
X = X.drop('surftype_j', 1)

X['near_angle'] = abs(X['near_angle'])

del X['bid_i']  # This has NaN and will cause errors in sklearn
del X['bid_j']

del X['side_i']
del X['side_j']
del X['length_i']
del X['length_j']
del X['sw_width_i']
del X['sw_width_j']

X.head()

In [ ]:
# Scaling appropriate features by subtracting mean and scaling to unit variance
scaler = preprocessing.StandardScaler()

X['near_distance'] = scaler.fit_transform(X['near_distance'])
X['length_diff'] = scaler.fit_transform(X['length_diff'])
X['sw_width_diff'] = scaler.fit_transform(X['sw_width_diff'])

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.5, random_state=6883)

In [ ]:
from sklearn import svm

#SVM with a RBF kernel 
svmR = svm.SVC(kernel='rbf')
svmR.fit(X_train, y_train)
print svmR.score(X_test, y_test)
svmR_pred = svmR.predict(X_test)
print precision_score(y_test, svmR_pred)
print recall_score(y_test, svmR_pred)

In [ ]:
X_pred = svmR.predict(X)

X_out = X.copy()
X_acc = X_pred == y

true_positives = X_out[np.logical_and(X_pred, X_acc)]
false_positives = X_out[np.logical_and(X_pred, np.logical_not(X_acc))]
true_negatives = X_out[np.logical_and(np.logical_not(X_pred), X_acc)]
false_negatives = X_out[np.logical_and(np.logical_not(X_pred), np.logical_not(X_acc))]

print true_positives.shape
print true_negatives.shape
print false_positives.shape
print false_negatives.shape

In [ ]:
# Export errors as geojson for visualizing on github
import json
from shapely import wkt
from shapely.geometry import mapping

geojson = {}
geojson['type'] = 'FeatureCollection'
geojson['features'] = []

for i in range(len(X_pred)):
    if not X_acc[i]:
        feature = {}
        feature['type'] = 'Feature'
        geom = wkt.loads(near_line[i])
        feature['geometry'] = mapping(geom)
        if X_pred[i]:
            properties = {'predicted': 1,
                          'type': 'false positive',
                          'stroke': '#FC0000',
                          'stroke-width': 5}
        else:
            properties = {'predicted': 0,
                          'type': 'false negative',
                          'stroke': '#093CA8',
                          'stroke-width': 5}
        feature['properties'] = properties
        geojson['features'].append(feature)
        
with open('./learndata-errorsCrowd.geojson', 'w') as f:
    json.dump(geojson, f)

In [ ]:
raw_data3 = pd.DataFrame()

True_disagree = 0. 
False_disagree = 0. 

NoConn = sum(raw_data['connected'])
NotConn = len(raw_data) - NoConn + 0. 

for i in range(0,len(crowd_labels)):
    temp = raw_data[raw_data['id_i']==crowd_labels['swi_id'][i]]
    temp2 = temp[temp['id_j']==crowd_labels['swj_id'][i]]
    raw_data3 = raw_data3.append(temp2,ignore_index=True)
    

    if (i%1000==0):
        print i

y3 = raw_data3['connected']        

In [ ]:
X_pred = y3

X_out = X.copy()
X_acc = X_pred == y

true_positives = X_out[np.logical_and(X_pred, X_acc)]
false_positives = X_out[np.logical_and(X_pred, np.logical_not(X_acc))]
true_negatives = X_out[np.logical_and(np.logical_not(X_pred), X_acc)]
false_negatives = X_out[np.logical_and(np.logical_not(X_pred), np.logical_not(X_acc))]

print true_positives.shape
print true_negatives.shape
print false_positives.shape
print false_negatives.shape


geojson = {}
geojson['type'] = 'FeatureCollection'
geojson['features'] = []

for i in range(len(X_pred)):
    if not X_acc[i]:
        feature = {}
        feature['type'] = 'Feature'
        geom = wkt.loads(near_line[i])
        feature['geometry'] = mapping(geom)
        if X_pred[i]:
            properties = {'predicted': 1,
                          'type': 'false positive',
                          'stroke': '#FC0000',
                          'stroke-width': 5}
        else:
            properties = {'predicted': 0,
                          'type': 'false negative',
                          'stroke': '#093CA8',
                          'stroke-width': 5}
        feature['properties'] = properties
        geojson['features'].append(feature)
        
with open('./learndata-errorsHeuristic_onCrowd.geojson', 'w') as f:
    json.dump(geojson, f)
